# 5단원

### 결정 트리

시각화가 매우 뛰어나다 

맨 위의 노드는 root node 맨 아래의 노드를 leaf node라고 한다

노드는 훈련 데이터의 특성에 대한 테스트를 표현한다

가지는 테스트의 결과(True, False)를 나타낸다

### 불순도

지니는 지니 불순도를 의미한다

지니 불순도 = 1-(음성클래스비율**2 + 양성클래스비율**2)

결정 트리 모델은 부모 노드와 자식 노드의 불순도 차이가 가능한 크도록 트리를 성장시킨다

부모와 자식 노드 사이의 불순도 차이를 정보 이득이라고 한다

criterion에 ‘entropy’ 또한 사용 가능하다 이를 지정하여 엔트로피 불순도를 사용할 수 있다

### 가지치기

가지치기를 위해서 max_depth 매개변수를 3으로 지정해보자!

이렇게 하면 루트 노드 아래로 최대 3개의 노드까지만 성장할 수 있다

### 교차 검증과 그리드 서치(어렵다)

테스트 세트로 일반화 성능을 올바르게 예측하려면 가능한 한 테스트 세트를 사용하지 말아야 한다

### 검증 세트

테스트 세트를 사용하지 않으면 모델이 과소적합인지 과대적합인지 판단하기 어렵다

테스트 세트를 사용하지 않고 이를 측정하는 간단한 방법은 훈련 세트를 또 나누는 것이다

이 데이터를 **검증 세트**라고 한다

### 교차 검증

[[기계학습] 교차검증(Cross Validation)](https://gsbang.tistory.com/entry/기계학습-교차검증Cross-Validation)

훈련 데이터가 많지 않을 때 사용한다. 

훈련 데이터로 학습하고 테스트 데이터로 평가하는 경우 해당 테스트 데이터에만 과적합되는 모델이 생성되어 일반화 성능이 떨어진다

훈련 데이터 (training data)에서 검증 데이터(validation data)를 떼어내어 모형을 검증하는 과정을 여러번 반복한다.

cross_validate 함수는 기본적으로 5-폴드 교차 검증을 수행한다 cv 매개변수에서 폴드 수를 바꿀 수 있다

scores 내의 test_score는 검증 폴드의 점수를 나타내므로 mean을 이용하여 평균 값을 계산한다

cross_validate함수는 훈련 세트를 섞어 폴드를 나누지 않는다.

만약 교차 검증을 할 때 훈련 세트를 섞으려면 splitter를 지정해 줘야 한다

cross_validate 함수는 회귀 모델일 경우 KFold 분할기를, 분류 모델일 경우 StratifiedKFold를 사용

StratifiedKFold는 target에 속성값의 개수를 동일하게 가져감으로써 kfold 같이 데이터가 한 곳으로 몰리는 것을 방지한다

### 하이퍼파라미터튜닝

매개 변수가 여러 개 일 때 하나의 매개변수의 최적값을 찾아 고정시켜놓고 다른 매개변수의 최적값을 찾는 것이 아니다. **두 매개변수를 동시에 바꿔가며 최적의 값을 찾아야 한다**

for 반복문으로 이런 과정을 직접 구현하기보다, **그리드 서치를 사용하자**

### GridSearchCV

**하이퍼파라미터 탐색과 교차검증을 한 번에 수행한다** ⇒ cross_validate() 함수를 호출할 필요가 없다

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(), params, n_jobs=-1)
gs.fit(train_input, train_target)
# 최적의 하이퍼파라미터를 찾으면 best_estimator_에 저장한다
dt = gs.best_estimator_
print(dt.score(train_input,train_target))
# 최적의 매개변수는 best_params_ 속성에 저장된다
print(gs.best_params_)
{'min_impurity_decrease': 0.0001}
#각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_results_속성의
# mead_test_score에 저장되어 있다
print(gs.cv_results_['mean_test_score'])

그러나 우리가 정해 주어야 하는 하이퍼파라미터 매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울 수 있다. 또 너무 많은 매개변수 조건이 있어 그리드 서치 수행 기간이 오래 걸릴 수 있다. 이럴 때 랜덤 서치를 사용하면 좋다

랜덤 서치는 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링 할 수 있는 확률 분포 객체를 전달한다.

In [ ]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

dt = gs.best_estimator_

print(dt.score(test_input, test_target))

## 트리의 앙상블

### 랜덤 포레스트

결정 트리를 랜덤하게 만들어 결정 트리의 숲을 만든다

랜덤 포레스트는 각 트리를 훈련하기 위한 데이터를 랜덤하게 만드는데, 우리가 입력한 훈련 데이터에서 랜덤하게 샘플을 추출하여 훈련 데이터를 만든다. 복원 추출 샘플을 **부트스트랩 샘플**

RandomForestClassifier는 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 선택한다.

RandomForestRegressor는 전체 특성을 사용한다

랜덤 포레스트는 랜덤하게 선택한 샘플과 특성을 사용하기 때문에 훈련 세트에 과대적합 되는것을 막아주고 검증 세트와 테스트 세트에서 안정적인 성능을 얻을 수 있다

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1,random_state=42)
scores = cross_validate(rf,train_input,train_target,return_train_score=True,n_jobs=-1)
print(scores)
#{'fit_time': array([0.52161193, 0.55875731, 0.50105882, 0.48093605, 0.52061486]), 
#'score_time': array([0.0618093 , 0.05036354, 0.07029223, 0.07935309, 0.06029129]), 
#'test_score': array([0.88461538, 0.88942308, 0.90279115, 0.88931665, 0.88642926]), 
#'train_score': array([0.9971133 , 0.99663219, 0.9978355 , 0.9973545 , 0.9978355 ])}

### 엑스트라 트리

부트스트랩 샘플을 사용하지 않고 각 결정 트리를 만들 때 전체 훈련 세트를 사용한다. 대신 노드를 분할할 때 무작위로 분할한다

많은 트리를 앙상블하기에 과대적합을 막고 검증 세트의 점수를 높인다